In [1]:
# https://www.kaggle.com/c/facial-keypoints-detection/data

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import keras as K

import tensorflow as tf
import os 
import sys
from datetime import datetime

from TrainImageObj import TrainImage # be careful with the tab vs ' '
# object used to debug and evaluate imported images
#CAREFUL: I did not handle nans, and in the presence of a nan an error occurs 

cwd = os.getcwd()
test = False
start = datetime.now()

projectDirectory = "C:\\Users\\Ted\\Projects\\Python\\FacialKeypointsDetection"
dataDir = "E:\\TK_PracticeDatabases\\facial-keypoints-detection"
os.chdir(dataDir)
testData = pd.read_csv("test.csv")
trainingData = pd.read_csv("training.csv")
idLookupData = pd.read_csv("IdLookupTable.csv")
sampleSubData = pd.read_csv("SampleSubmission.csv")

os.chdir(cwd)

if test:
    print(trainingData)

importComplete = datetime.now()
print("Import Finished: " + str(importComplete - start))

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Import Finished: 0:00:04.584662


In [79]:
os.chdir(projectDirectory)
vgg16 = VGG16(FC_Include=True)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="block1_conv1", padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:134: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu", name="block5_conv2", padding="same")`


In [78]:
def VGG16(ClassicVGG16=True, 
          FC_Include = True,
          classificationNumber = 1000,
          l2_weight = 5e-04,
          like_its_hot = 0.7, # drop regulation
          FeatureExtractorTraining = False, 
          FCTraining = True,
          weights= 'imagenet', 
          input_tensor=None): 
    ''' 
    # Inputs
    FC_Include = using the network as a feature extractor based on the convolutional layers
    FullyConnected = if training is needed for the fully connected layer
    classificationNumber = number of components 
    FeatureExtractorTraining = if you need to train the middle layers
    weights = 'imagenet' means using the weights from a network pretrained by imagenet challenge
    
    Rules:
    - CANNOT have MORE than 1000 outputs (can't really see a case where they're would be >1k but eat your heart out.
    - Use the excess and keep training on the FC layers true
    - Need to append additional layers of lower training weights if want to use the non-FC layers
    
    # Returns
        VGG16 Network
    '''
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')
    # Determine proper input shape
    if ClassicVGG16:
        inputShape = (224, 224, 3)
    else:
        inputShape = (None, None, 3)
    
    img_input = K.Input(inputShape)
    
    # Block 1     
    b1_1 = K.layers.Conv2D(64, (3, 3), 
                  activation='relu', 
                  border_mode='same', 
                  name='block1_conv1')
    b1_1.trainable = FeatureExtractorTraining
    x = b1_1(img_input)
    
    b1_2 = K.layers.Conv2D(64, (3, 3), 
                  activation='relu', 
                  padding='same', 
                  name='block1_conv2')
    b1_2.trainable = FeatureExtractorTraining
    x = b1_2(x)#_normalized)
    
    x = K.layers.MaxPooling2D((2,2), strides=(2,2), name='block1_pool')(x) 
    
    # Block 2
    b2_1 = K.layers.Conv2D(128, 
                  (3, 3), 
                  activation='relu', 
                  padding='same', 
                  name='block2_conv1')
    b2_1.trainable = FeatureExtractorTraining
    x = b2_1(x)
    
    b2_2 = K.layers.Conv2D(128, (3, 3), 
                  activation='relu', 
                  padding='same', 
                  name='block2_conv2')
    b2_2.trainable = FeatureExtractorTraining
    x = b2_2(x)#_normalized)
    
    x = K.layers.MaxPooling2D((2,2), strides=(2,2) , name='block2_pool')(x)#_normalized) # decrease the amout of data points with no rounding loss
    
    # Block 3
    # convolution block
    
    b3_1 = K.layers.Conv2D(256, (3, 3), 
                  activation='relu', 
                  padding='same', 
                  name='block3_conv1')
    b3_1.trainable = FeatureExtractorTraining
    x = b3_1(x)
    
    b3_2 = K.layers.Conv2D(256, (3, 3), 
                  activation='relu', 
                  padding='same', 
                  name='block3_conv2')
    b3_2.trainable = FeatureExtractorTraining
    x = b3_2(x)#_normalized)

    b3_3 = K.layers.Conv2D(256, (3, 3), 
                  activation='relu', 
                  padding='same', 
                  name='block3_conv3')
    b3_3.trainable = FeatureExtractorTraining
    x = b3_3(x)#_normalized)
    
    x = K.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
    
    # Block 4 identity doc
    
    b4_1 = K.layers.Conv2D(512, (3, 3), 
                  activation='relu', 
                  padding='same', 
                  name='block4_conv1')
    b4_1.trainable = FeatureExtractorTraining
    x = b4_1(x)
    
    b4_2 = K.layers.Conv2D(512, (3, 3), 
                  activation='relu', 
                  padding='same', 
                  name='block4_conv2')
    b4_2.trainable = FeatureExtractorTraining
    x = b4_2(x)#_normalized)

    b4_3 = K.layers.Conv2D(512, (3, 3), 
                  activation='relu', 
                  padding='same', 
                  name='block4_conv3')
    b4_3.trainable = FeatureExtractorTraining
    x = b4_3(x)#_normalized)
    
    x = K.layers.MaxPooling2D((2,2), strides=(2,2), name='block4_pool')(x)
    
    #Block 5
    b5_1 = K.layers.Conv2D(512, (3, 3), 
                  activation='relu', 
                  padding='same', 
                  name='block5_conv1')
    b5_1.trainable = FeatureExtractorTraining
    x = b5_1(x)
    
    b5_2 = K.layers.Conv2D(512, (3, 3), 
                  activation='relu', 
                  border_mode='same', 
                  name='block5_conv2')
    b5_2.trainable = FeatureExtractorTraining
    x = b5_2(x)
    
    b5_3 = K.layers.Conv2D(512, (3, 3), 
                  activation='relu', 
                  padding='same', 
                  name='block5_conv3')
    b5_3.trainable = FeatureExtractorTraining
    x = b5_3(x)
    
    x = K.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if FC_Include:
        # Classification block
        x = K.layers.Flatten(name='flatten')(x)
        
        #x = Dropout(like_its_hot, name = 'regulator_0')(x)
        fc1 = K.layers.Dense(4096, activation='relu',
                    kernel_regularizer=K.regularizers.l2(l2_weight),
                    name='fc1')
        fc1.trainable = FCTraining
        x = fc1(x)
        
        x = K.layers.Dropout(like_its_hot, name = 'regulator_1')(x)
        
        fc2 = K.layers.Dense(4096, 
                    activation='relu', 
                    kernel_regularizer=K.regularizers.l2(l2_weight),
                    name='fc2')
        fc2.trainable = FCTraining
        x = fc2(x)
        
        x = K.layers.Dropout(like_its_hot, name = 'regulator_2')(x)
        
        pred  = K.layers.Dense(classificationNumber, 
                      activation='softmax', 
                      kernel_regularizer=K.regularizers.l2(l2_weight),
                      name='pred')(x)
        
        model = K.Model(img_input, pred)
        
    else: ########################################################################################
        print ("You got no legs Lieutenant Dan!!!")
        model = K.Model(img_input,x)
        
    # load weights
    if weights == 'imagenet':
        currentCwd = os.getcwd()
        os.chdir(projectDirectory) # hard coded for my directory
        if FC_Include == False:
            modelWeights = model.load_weights('vgg16Weights_noFC.h5')            
        elif FC_Include == True: # only include the top if has 
            modelWeights = model.load_weights('vgg16Weights_FCincluded.h5')
        os.chdir(currentCwd)
    return model

In [18]:
def weighted_binary_crossentropy(y_true, y_pred):
    # Oiginal binary crossentropy (see losses.py):
    # K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1)
    # Calculate the binary crossentropy
    
    one_weight = 0.75
    zero_weight = 1.0 - one_weight
    b_ce = K.binary_crossentropy(y_true, y_pred)
    # Apply the weights
    weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
    weighted_b_ce = weight_vector * b_ce
    # Return the mean error
    return K.mean(weighted_b_ce)

# Defining Arcitecture for Neural Net
## OBJ
- __init__
    - defines functions that can be called only 
- First Function
    - VGG16 sequentially network
    - boolean on training CNN layers
    - boolean for using the CNN layers as feature extractor
    - boolean on training Neural Net layers
    - weight to the 1s 
        - if =np.nan then weights are .5 and .5 for weighted X-entropy
    - Output: full architecture
- Second Function
    - Training Layer
    - Output: Fully Trained Model
- Third Function
    - Testing Layer
    - Output: Model to be used on Data
    
# Defining Arcitecture for Neural Net
## OBJ
- __init__
    - defines functions that can be called only 
- First Function
    - K fold cross validation
    - input 2 ints 
        - number of 0s
        - number of 1s
            - if only the first one populated then force weighted k fold to be false
    - number of bins
        - if bins = np.nan, then calculate even number of bins, if %bins  less than bins then add to final  
    - boolean for weighted k fold (evenly distributed) making even 1s in each bin
        - just incase unweighted 0s and 1s
        - separate the input vector into 2
    - Output: bins that are itterable


In [2]:
# save all images that are not labeled and labeled until a nan appears 
# for the first time 

imageDir = "SubplotImages"
os.chdir(imageDir)
for i in range(0,209):#,len(trainingData)): # only need so many (Nans make anything beyond this tricky to say the least)
    patient = TrainImage(trainingData, i) # load the images, labeled and not labeled
    plt.subplot(121)
    plt.imshow(patient.image)
    plt.title(str(i)+" Base")
    plt.subplot(122)
    plt.imshow(patient.labeledImage)
    plt.title(str(i)+" Labeled")
    plt.savefig(str(i)+" Base&Labeled")
    plt.close()
os.chdir(cwd)

C:\Users\Ted\Projects\Python\FacialKeypointsDetection\TrainImageObj.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  trainData["left_eye_center_y"][index] = 0
C:\Users\Ted\Projects\Python\FacialKeypointsDetection\TrainImageObj.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  trainData["left_eye_center_x"][index] = 0
C:\Users\Ted\Projects\Python\FacialKeypointsDetection\TrainImageObj.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  trainData["left_eye_inner_corner_y"][index]

KeyboardInterrupt: 